In [5]:
# _*_coding:utf-8 _*_
# @Time　　:2020/2/29/21:32
# @Author　 : Space-time network research group
#Contact Simon Xuesong Zhou at xzhou74@asu.edu 
#@ File　　  :generic label correcting algorithm.py
#@Software  :PyCharm

"""import package"""
import pandas as pd
import numpy as np
import copy

In [6]:
"""define global variables"""
g_node_list=[] #set of node
g_node_zone={} #set of node type
g_link_list=[] #set of arc
g_shortest_path=[]  #set of shortest path
g_origin=None       #source node of network
g_number_of_nodes=0 #node number
node_predecessor =[] #set of predecessor node
node_label_cost  =[]  #set of distance label
Max_label_cost   =99999#initial distance label

"""import network data file and initialize corresponding variables"""
#read data of nodes
df_node=pd.read_csv('node.csv')
df_node=df_node.iloc[:,:].values
for i in range(len(df_node)):
    g_node_list.append(df_node[i,0])
    g_node_zone[df_node[i,0]]=df_node[i,-1]
    g_number_of_nodes+=1
    
    print ("Set of node: ", g_node_list)
    print ("Set of node zone: ", g_node_zone)
    
    if df_node[i,3]==1:
        g_origin=df_node[i,0]
        
    print("The origin node is: ", g_origin)

Set of node:  [1]
Set of node zone:  {1: 1}
The origin node is:  1
Set of node:  [1, 2]
Set of node zone:  {1: 1, 2: 2}
The origin node is:  1
Set of node:  [1, 2, 3]
Set of node zone:  {1: 1, 2: 2, 3: 3}
The origin node is:  1
Set of node:  [1, 2, 3, 4]
Set of node zone:  {1: 1, 2: 2, 3: 3, 4: 4}
The origin node is:  1
Set of node:  [1, 2, 3, 4, 5]
Set of node zone:  {1: 1, 2: 2, 3: 3, 4: 4, 5: 5}
The origin node is:  1
Set of node:  [1, 2, 3, 4, 5, 6]
Set of node zone:  {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6}
The origin node is:  1


In [7]:
Distance=np.ones((g_number_of_nodes, g_number_of_nodes))*Max_label_cost #distance matrix
node_predecessor=[-1]*g_number_of_nodes
node_label_cost=[Max_label_cost]*g_number_of_nodes
node_predecessor[g_origin-1]=0
node_label_cost[g_origin-1]=0
#read data of arcs
df_link=pd.read_csv('link.csv')
df_link=df_link.iloc[:,:].values
for i in range(len(df_link)):
    g_link_list.append((df_link[i, 1],df_link[i, 2]))
    Distance[df_link[i, 1]-1, df_link[i, 2]-1]=df_link[i, 3]
print()
print("Updating the distance matrix using link.csv")
print (Distance)


Updating the distance matrix using link.csv
[[9.9999e+04 6.0000e+00 4.0000e+00 9.9999e+04 9.9999e+04 9.9999e+04]
 [9.9999e+04 9.9999e+04 2.0000e+00 2.0000e+00 9.9999e+04 9.9999e+04]
 [9.9999e+04 9.9999e+04 9.9999e+04 1.0000e+00 2.0000e+00 9.9999e+04]
 [9.9999e+04 9.9999e+04 9.9999e+04 9.9999e+04 9.9999e+04 7.0000e+00]
 [9.9999e+04 9.9999e+04 9.9999e+04 1.0000e+00 9.9999e+04 3.0000e+00]
 [9.9999e+04 9.9999e+04 9.9999e+04 9.9999e+04 9.9999e+04 9.9999e+04]]


In [8]:
"""finding optimal solution: scan arc and check its optimality condition"""
iteration = 0 
while True:
    v=0 #number of arcs violating the optimality conditons
    for head,tail in g_link_list:
        if  node_label_cost[tail-1]>node_label_cost[head-1]+Distance[head-1,tail-1]:
            node_label_cost[tail-1]=node_label_cost[head-1]+Distance[head-1,tail-1]
            node_predecessor[tail-1]=head
            print ("iteration=", iteration)
            print ("node_label_cost=", node_label_cost)
            print ("node_predecessor=", node_predecessor)
            iteration = iteration +1
            v=v+1
    if v==0:
        break

iteration= 0
node_label_cost= [0, 6.0, 99999, 99999, 99999, 99999]
node_predecessor= [0, 1, -1, -1, -1, -1]
iteration= 1
node_label_cost= [0, 6.0, 4.0, 99999, 99999, 99999]
node_predecessor= [0, 1, 1, -1, -1, -1]
iteration= 2
node_label_cost= [0, 6.0, 4.0, 8.0, 99999, 99999]
node_predecessor= [0, 1, 1, 2, -1, -1]
iteration= 3
node_label_cost= [0, 6.0, 4.0, 5.0, 99999, 99999]
node_predecessor= [0, 1, 1, 3, -1, -1]
iteration= 4
node_label_cost= [0, 6.0, 4.0, 5.0, 6.0, 99999]
node_predecessor= [0, 1, 1, 3, 3, -1]
iteration= 5
node_label_cost= [0, 6.0, 4.0, 5.0, 6.0, 12.0]
node_predecessor= [0, 1, 1, 3, 3, 4]
iteration= 6
node_label_cost= [0, 6.0, 4.0, 5.0, 6.0, 9.0]
node_predecessor= [0, 1, 1, 3, 3, 5]


In [9]:
"""generate shortest paths according to the predecessor node"""
agent_id=1
o_zone_id=g_node_zone[g_origin]
for destination in g_node_list:
    if g_origin!=destination:
        d_zone_id=g_node_zone[destination]
        if node_label_cost[destination-1]==Max_label_cost:
            path=" "
            g_shortest_path.append([agent_id,o_zone_id,d_zone_id,path,node_label_cost[destination-1]])
        else:
            to_node=copy.copy(destination)
            path="%s"%to_node
            while node_predecessor[to_node-1]!=g_origin:
                path="%s;"%node_predecessor[to_node-1]+path
                g=node_predecessor[to_node-1]
                to_node=g
            path="%s;"%g_origin+path
            g_shortest_path.append([agent_id,o_zone_id,d_zone_id, path, node_label_cost[destination - 1]])
            print('from {} to {} the path is {}，length is {}'
                      .format(g_origin,destination,path,node_label_cost[destination-1]))
        agent_id+=1

from 1 to 2 the path is 1;2，length is 6.0
from 1 to 3 the path is 1;3，length is 4.0
from 1 to 4 the path is 1;3;4，length is 5.0
from 1 to 5 the path is 1;3;5，length is 6.0
from 1 to 6 the path is 1;3;5;6，length is 9.0


In [10]:
"""output result into csv file"""
#transform data into DataFrame
g_shortest_path=np.array(g_shortest_path)
col=['agent_id','o_zone_id','d_zone_id','node_sequence','distance']
file_data = pd.DataFrame(g_shortest_path, index=range(len(g_shortest_path)),columns=col)
file_data.to_csv('agent.csv',index=False)